# Interaktív térképek Folium-mal

Ebben a notebookban három, információban gazdag, interaktív térképet készítettem a Folium könyvtárral.
Külső adatfájlokat nem használok, a szükséges adatok be vannak ágyazva a notebookba.

- Európai fővárosok: MarkerCluster, gazdag popupok, egyedi ikonok
- Budapest környéki pontsűrűség: HeatMap réteg
- Rétegek és vezérlők: több csemperéteg, népességarányos jelölők, útvonal, MiniMap, MeasureControl, LocateControl, LayerControl


In [15]:
import folium
from folium import plugins
import random

# ezres tagolás
def fmt_int(n):
    return f"{n:,}".replace(',', ' ') 


## 1) Európai fővárosok – MarkerCluster, gazdag popupok
Az alábbi térkép európai fővárosokat jelenít meg, marker-klaszterezéssel. A popup tartalmazza az országot,
a becsült lakosságot és földrajzi koordinátákat. A jelölő ikonja színezést kap a lakosságszint szerint.


In [16]:
capitals = [
    # name, country, lat, lon, population (approx)
    ('Budapest', 'Hungary', 47.4979, 19.0402, 1_750_000),
    ('Vienna', 'Austria', 48.2082, 16.3738, 1_900_000),
    ('Bratislava', 'Slovakia', 48.1486, 17.1077, 475_000),
    ('Prague', 'Czechia', 50.0755, 14.4378, 1_320_000),
    ('Warsaw', 'Poland', 52.2297, 21.0122, 1_790_000),
    ('Berlin', 'Germany', 52.5200, 13.4050, 3_660_000),
    ('Copenhagen', 'Denmark', 55.6761, 12.5683, 800_000),
    ('Stockholm', 'Sweden', 59.3293, 18.0686, 975_000),
    ('Oslo', 'Norway', 59.9139, 10.7522, 700_000),
    ('Helsinki', 'Finland', 60.1699, 24.9384, 655_000),
    ('Paris', 'France', 48.8566, 2.3522, 2_140_000),
    ('Madrid', 'Spain', 40.4168, -3.7038, 3_220_000),
    ('Lisbon', 'Portugal', 38.7223, -9.1393, 505_000),
    ('Rome', 'Italy', 41.9028, 12.4964, 2_870_000),
    ('Athens', 'Greece', 37.9838, 23.7275, 664_000),
    ('Amsterdam', 'Netherlands', 52.3676, 4.9041, 872_000),
    ('Brussels', 'Belgium', 50.8503, 4.3517, 1_850_000),
    ('Dublin', 'Ireland', 53.3498, -6.2603, 554_000),
    ('Zurich', 'Switzerland', 47.3769, 8.5417, 434_000),
]

m1 = folium.Map(location=[50.0, 14.0], zoom_start=5, tiles='CartoDB positron', control_scale=True)
cluster = plugins.MarkerCluster(name='Fővárosok').add_to(m1)

for name, country, lat, lon, pop in capitals:
    # ikon színe a népesség szerint
    color = 'green' if pop < 700_000 else ('orange' if pop < 1_500_000 else 'red')
    popup_html = folium.Html(f"""
        <b>{name}</b><br/>
        Ország: {country}<br/>
        Népesség: {fmt_int(pop)} fő<br/>
        Koordináták: {lat:.4f}, {lon:.4f}
    """, script=True)
    folium.Marker(
        [lat, lon],
        icon=folium.Icon(color=color, icon='info-sign'),
        tooltip=f'{name} – {country}',
        popup=folium.Popup(popup_html, max_width=250)
    ).add_to(cluster)

m1


Következtetés: A jelölők klaszterezése segít áttekinteni a sűrűn elhelyezkedő városokat. A lakosságszínkódolás (zöld–narancs–piros) azonnali benyomást ad a városok méretéről.


## 2) Budapest és környéke – pontsűrűség HeatMap
Véletlenszerű (reprodukálható) pontokat generálunk Budapest környékén, és sűrűségtérképen ábrázoljuk őket.
A példában a pontok több mag körül koncentrálódnak (belváros, Buda, pesti peremkerületek).


In [17]:
random.seed(42)
bp_center = (47.4979, 19.0402)
centers = [
    (47.4979, 19.0402, 300),   # belváros
    (47.5090, 19.0250, 150),   # Buda
    (47.5200, 19.1000, 100),   # pesti oldal
]
points = []
for lat0, lon0, n in centers:
    for _ in range(n):
        # ~1-3 km-es szórás körülbelül: 0.01-0.03 fok lat/lon
        lat = random.gauss(lat0, 0.01)
        lon = random.gauss(lon0, 0.015)
        weight = random.uniform(0.5, 1.0)
        points.append([lat, lon, weight])

m2 = folium.Map(location=bp_center, zoom_start=12, tiles='CartoDB dark_matter', control_scale=True)
plugins.HeatMap(points, name='Pontsűrűség', min_opacity=0.3, radius=40, blur=25, max_zoom=18).add_to(m2)
folium.LayerControl(collapsed=False).add_to(m2)
m2


## 3) Rétegek és vezérlők – népességarányos jelölők, útvonal, MiniMap
Ebben a példában több réteget kombinálunk: különböző térképfajták, népességarányos körjelölők, egy rövid útvonal,
és hasznos vezérlők (rétegkapcsoló, távolságmérés, pozíció meghatározás, áttekintő térkép).


In [18]:
m3 = folium.Map(location=[48.2, 17.0], zoom_start=6, tiles=None, control_scale=True)
# Csemperétegek
folium.TileLayer('OpenStreetMap', name='OSM').add_to(m3)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}',
    attr='Tiles © Esri — Sources: USGS, NOAA',
    name='Domborzat (Esri)'
).add_to(m3)
folium.TileLayer('CartoDB positron', name='Világos').add_to(m3)
folium.TileLayer('CartoDB dark_matter', name='Sötét').add_to(m3)

# Népességarányos körök rétege
cities_layer = folium.FeatureGroup(name='Városok (körjelölők)', show=True)
for name, country, lat, lon, pop in capitals:
    radius = max(4, min(30, pop / 150000))  # skálázás
    folium.CircleMarker(
        [lat, lon],
        radius=radius,
        color='#2b8cbe', fill=True, fill_color='#7bccc4', fill_opacity=0.6,
        tooltip=f'{name} ({country}) – {fmt_int(pop)} fő'
    ).add_to(cities_layer)
cities_layer.add_to(m3)

# Útvonal (Példa: Bécs → Pozsony → Budapest)
route = [
    [48.2082, 16.3738],  # Vienna
    [48.1486, 17.1077],  # Bratislava
    [47.4979, 19.0402],  # Budapest
]
folium.PolyLine(route, color='orange', weight=4, opacity=0.8, tooltip='Bécs → Pozsony → Budapest').add_to(m3)

# Vezérlők
plugins.MiniMap(toggle_display=True).add_to(m3)
plugins.MeasureControl(position='topleft', primary_length_unit='kilometers').add_to(m3)
plugins.LocateControl(auto_start=False, keepCurrentZoomLevel=True).add_to(m3)
folium.LayerControl(collapsed=False).add_to(m3)

m3
